# Лабораторная работа по теме "Metamorphic testing"

Ссылка на материалы занятия https://drive.google.com/drive/folders/1dbq1scqU22XPFCrVmz75gpRsb49QjtcM?usp=drive_link

Литература

1) Barr E.T., Harman M., McMinn P., Shahbaz M., Yoo S. The oracle problem in software testing: A survey. IEEE transactions on software engineering. 41(5). 2014. P. 507-525. https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6963470

2) Tsong Yueh Chen, Fei-Ching Kuo, Huai Liu, Pak-Lok Poon, Dave Towey, T. H.
Tse, and Zhi Quan Zhou. 2018. Metamorphic Testing: A Review of Challenges
and Opportunities. 51, 1, Article 4 (Jan. 2018), 27 pages. https://doi.org/10.1145/3143561

3) M. Srinivasan, M. P. Shahri, I. Kahanda and U. Kanewala, "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing", 2018 IEEE/ACM 3rd International Workshop on Metamorphic Testing (MET), Gothenburg, 2018, pp. 26-33. https://arxiv.org/pdf/1802.07354.pdf


**Критерии оценки.**

Условная оценка - 10 баллов за лабораторную. Для получения зачёта достаточно набрать 6.



- Задание 1. (4 вопроса, 1 балл)
- Задание 2. (9 баллов)

## Задание 1. Теоретическая часть

Ответьте на следующие вопросы (*1 балл*).
1. Опишите суть проблемы тестового оракула.
2. Приведите примеры задач, для которых обычное тестирование с оракулом не подходит.
3. Перечислите методы, которые применяются для решения этой проблемы.
4. Дайте определение тестового инварианта (metamorphic relation).





Приведите свои ответы здесь:

1. Чтобы првоерить ответы программы, нужно чтобы ответ уже был, либо легко проверяем.
2. Например, задачи, связанные с весами нейронной сети. Задачи по генерации текста.
3. Implicit oracles, comparison testing, fuzzing, metamorphic testing, assertion testing.
4. Свойство, которое должно сохраняться при изменении входных данных программы, если она работает корректно. Иногда сложно проверить правильность результата напрямую, но можно изменить вход (например, поменять порядок предложений, изменить регистр букв, добавить шум) и заранее знать, как должен измениться (или не измениться) результат.

## Задание 2. Разработка тестовых инвариантов

Дана модель для распознавания сущностей в тексте.
- Придумайте и реализуйте 2 теста с тестовым оракулом (обычные тесты с правильными ответами) (*1 балл*)
- Придумайте и реализуйте не менее 3 тестовых инвариантов (metamorphic relations) для её проверки - (*суммарно 6 баллов, теоретическое описание - по 1 баллу, реализация - по 1 баллу*)
- Сравните полученные тесты. В чем преимущества каждого из методов? В чём недостатки? (*2 балла*)

*Указание*. Можете воспользоваться идеями со слайда "Bio-entity recognition" или из статьи "Quality Assurance of Bioinformatics Software: A Case Study of Testing a Biomedical Text Processing Tool Using Metamorphic Testing" из списка литературы.

**Правила оформления.**

Для каждого инварианта необходимо описать
 - из каких предположений о модели он вытекает,
 - формальное описание (желательно с формулой),
 - запуск на 1-2 примерах тестовых данных.

Теоретическую часть можно оформить в ячейке markdown.

### Тестируемая система

Модель позволяет искать сущности в тексте. Ниже приведены примеры того, как можно с ней работать.

In [1]:
# ! pip install spacy==2.2.4

In [2]:
import spacy
from spacy import displacy

In [3]:
# example of model usage
def render(text):

    nlp = spacy.load('en_core_web_sm') # model

    doc = nlp(text) # data processing

    # FYI all the properties
    props = [p for p in dir(doc.ents[0]) if p[0] != '_']
    print(props)

    # custom processing of the answer

    # get counts of entities
    ent_labels = [e.label_ for e in doc.ents]
    freq = dict()
    for l in ent_labels:
        freq[l] = ent_labels.count(l)
    print(freq)

    # get coordinates of entities
    coordinates = [e.start_char for e in doc.ents]
    print(coordinates)

    # render the answer
    displacy.render(doc, style='ent', jupyter=True)


Модель позволяет искать сущности в тексте. В ячейке ниже приведены примеры того, как можно с ней работать.

In [4]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

render(text)

['as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'id', 'id_', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'sents', 'set_extension', 'similarity', 'start', 'start_char', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'vector', 'vector_norm', 'vocab']
{'ORG': 4, 'LOC': 1}
[4, 84, 111, 124, 167]



### Пример оформления инварианта

Рассмотрим задачу поиска подстроки в строке.

Предполагаем, что алгоритм должен находить все вхождения подстроки.

> **MR1.** Если в строке S найдёна некоторая подстрока s ровно k раз, то в строке SS она будет найдена не менее 2k раз (возможны нахождения на месте склейки строк).

Формально. Пусть S - строка, s - её подстрока, f(S,s) - определённое программой число вхождений s в S. Тогда f(SS,s) >= 2*f(S,s).


In [5]:
import re
import unittest

# function for testing
def func_to_test(substr, string):
    return re.finditer(pattern=substr, string=string)


class TestStringMethods(unittest.TestCase):

    def test_with_oracle1(self):
        # input data
        big_string = 'abacaba'
        substr = 'aba'

        # correct answer
        right_answer = [0, 4]

        indices = func_to_test(substr, big_string)
        answer = [index.start() for index in indices]

        self.assertTrue(answer == right_answer)

    def test_metamorphic1(self):
        # input data
        big_string1 = 'abacab'
        big_string2 = big_string1 + big_string1
        substr = 'aba'

        # first answer
        indices = func_to_test(substr, big_string1)
        indices1 = [index.start() for index in indices]

        # second answer
        indices = func_to_test(substr, big_string2)
        indices2 = [index.start() for index in indices]

        # metamorphic relation
        self.assertTrue(2*len(indices1) <= len(indices2))

Ниже мы запускаем все объявленные тестовые случаи

In [6]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


### Реализуйте собственные тестовые инварианты ниже

In [7]:
# для корректного вывода результатов
def get_result(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    props = [p for p in dir(doc.ents[0]) if p[0] != '_']
    ent_labels = [e.label_ for e in doc.ents]
    freq = dict()
    for l in ent_labels:
        freq[l] = ent_labels.count(l)
    coordinates = [e.start_char for e in doc.ents]
    return (freq, coordinates)

In [8]:
# поулчение по апи случайных шуток
import requests
import warnings
import urllib3

# отключить только предупреждение InsecureRequestWarning (вызываю без ссл)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_random_joke():
    url = "https://api.chucknorris.io/jokes/random"
    r = requests.get(url, verify=False)  # если SSL не настроен
    r.raise_for_status()
    return r.json()["value"]

print(get_random_joke())

You know why Chuck Norris-movies suck? Because the movies are nervous about Chuck Norris in 'em.


In [9]:
# синтез текста с разными форматами дат
import random
import datetime


def get_random_text_with_dates(n_sentences: int = 5) -> str:
    date_formats = [
        "%B %d, %Y",    
        "%d %B %Y",     
        "%Y-%m-%d",     
        "%d/%m/%Y",     
        "%m/%d/%y",     
        "%B %Y",        
        "%Y"            
    ]

    verbs = ["met", "visited", "founded", "was born on", "launched", "signed"]
    subjects = ["John", "Mary", "Tesla", "NASA", "Apple Inc.", "the company"]
    places = ["in New York", "in Paris", "at the headquarters", "in Berlin", "in Tokyo"]

    sentences = []
    for _ in range(n_sentences):
        dt = datetime.date.today() - datetime.timedelta(days=random.randint(0, 20000))
        date_str = dt.strftime(random.choice(date_formats))
        sentence = f"{random.choice(subjects)} {random.choice(verbs)} {random.choice(places)} on {date_str}."
        sentences.append(sentence)

    return " ".join(sentences)\


print(get_random_text_with_dates())

the company signed in Berlin on March 1976. Apple Inc. was born on at the headquarters on 25/11/2013. Mary founded at the headquarters on April 1993. the company visited in Berlin on 1979. NASA visited in Tokyo on October 25, 2000.


In [10]:
# поулчение текста с рандомными предложениями
def get_mixed_text(jokes_count: int = 3, date_sentences_count: int = 2) -> str:
    parts = []
    for _ in range(jokes_count):
        parts.append(get_random_joke())
    parts.append(get_random_text_with_dates(date_sentences_count))
    random.shuffle(parts)
    return " ".join(parts)

print(get_mixed_text())

Chuck Norris can make you lick your own elbow by twisting you into a pretzel. Chuck Norris was at a bar when a drunken guy called him a pussy. Chuck punched him so hard his kidneys shut down. Tesla visited in Paris on 12/16/79. Tesla was born on in Tokyo on 1999. Chuck Norris is a fat and lonely man who has bum sex with his 10 year old son, I know this because I am Chuck Norris son "dad get that small dick out of there"


### Тестирование

## 1 Инвариант:

Если склеить текст с самим собой через точку с пробелами как явный разделитель предложения ("` . `"), счётчик категории "DATE" должен ровно удвоиться.
Разделитель разрывает любые потенциальные слияния и не создаёт новых сущностей.

Т.е. cat1 = get_result(s), cat2 = get_result(s + "` . `" + s) => cat1["DATE"] * 2 = cat2["DATE"]

## 2 Инвариант

Если к тексту добавить одну сущность (например, дополнительную дату), то количество распознанных сущностей должно увеличиться ровно на 1.
  
Т. е. cat1 = get_result(s), cat2 = get_result(s + " on 2023-12-31.") => sum(cat1.values()) + 1 = sum(cat2.values())

## 3 Инвариант

Если к тексту добавить нейтральный текст, то количество распознанных сущностей не должно измениться.

Т. е. cat1 = get_result(s), cat2 = get_result(s + " lol kek") => sum(cat1.values()) = sum(cat2.values())

In [11]:
INVS_NUM = 10


import unittest

class TestStringMethods(unittest.TestCase):

    def test_with_oracle1(self):
        test_text = """One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.
        He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections."""

        result = get_result(test_text)
        correct_result = ({'TIME': 1, 'PERSON': 1}, [0, 18])

        self.assertTrue(result == correct_result)


    def test_with_oracle2(self):
        test_text = """The quick, brown fox jumps over a lazy dog. DJs flock by when MTV ax quiz prog. Junk MTV quiz graced by fox whelps.
        Bawds jog, flick quartz, vex nymphs. Waltz, bad nymph, for quick jigs vex! Fox nymphs grab quick-jived waltz.
        Brick quiz whangs jumpy veldt fox. Bright vixens jump; dozy fowl quack. Quick wafting zephyrs vex bold Jim.
        Quick zephyrs blow, vexing daft Jim. Sex-charged fop blew my junk TV quiz. How quickly daft jumping zebras vex.
        Two driven jocks help fax my big quiz."""

        result = get_result(test_text)
        correct_result = ({'ORG': 3, 'PERSON': 5, 'CARDINAL': 1}, [62, 85, 124, 161, 199, 242, 345, 390, 478])

        self.assertTrue(result == correct_result)


    def test_metamorphic1(self):
        def simple_test():
            s = get_mixed_text(3, 2)
            base = get_result(s)[0]
            dup = get_result(s + " . " + s)[0]
    
            base_date = base.get("DATE", 0)
            dup_date = dup.get("DATE", 0)
    
            self.assertEqual(dup_date, 2 * base_date)
    
        for _ in range(INVS_NUM):
            simple_test()
            print("%d test for 1 inv passed" % (_ + 1))


    def test_metamorphic2(self):
        def simple_test():
            base_text = get_mixed_text(3, 2)
            base = get_result(base_text)[0]
            base_count = sum(base.values())
    
            # Добавляем одну новую дату
            new_text = base_text + " on 2023-12-31."
            new = get_result(new_text)[0]
            new_count = sum(new.values())
    
            self.assertEqual(new_count, base_count + 1)
    
        for _ in range(INVS_NUM):
            simple_test()
            print("%d test for 2 inv passed" % (_ + 1))


    def test_metamorphic3(self):
        def simple_test():
            base_text = get_mixed_text(3, 2)
            base = get_result(base_text)[0]
            base_count = sum(base.values())
    
            # Добавляем одну новую дату
            new_text = base_text + " lol kek"
            new = get_result(new_text)[0]
            new_count = sum(new.values())
    
            self.assertEqual(new_count, base_count)
    
        for _ in range(INVS_NUM):
            simple_test()
            print("%d test for 3 inv passed" % (_ + 1))

In [12]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.chucknorris.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1 test for 1 inv passed
2 test for 1 inv passed
3 test for 1 inv passed
4 test for 1 inv passed
5 test for 1 inv passed
6 test for 1 inv passed
7 test for 1 inv passed
8 test for 1 inv passed
9 test for 1 inv passed


.

10 test for 1 inv passed
1 test for 2 inv passed
2 test for 2 inv passed
3 test for 2 inv passed
4 test for 2 inv passed
5 test for 2 inv passed
6 test for 2 inv passed
7 test for 2 inv passed
8 test for 2 inv passed
9 test for 2 inv passed


.

10 test for 2 inv passed
1 test for 3 inv passed
2 test for 3 inv passed
3 test for 3 inv passed
4 test for 3 inv passed
5 test for 3 inv passed
6 test for 3 inv passed
7 test for 3 inv passed
8 test for 3 inv passed
9 test for 3 inv passed


.

10 test for 3 inv passed


..
----------------------------------------------------------------------
Ran 5 tests in 114.491s

OK


## Результаты

Тест с оракулом дает однозначное представление о поведении функции. Однако у него есть недостаток - нужно иметь сами правильные ответы.

Тест с инвариантом позволяет проверять различные гипотезы на больших объемах данных без реальных ответов - все зависит от гипотезы, условия, которое мы предъявляем к функции. Однако это все еще не дает безошибочных результатов (во время тестирования, столкнулся с проблемой инвариантности имен для ```s.lower()```, и еще узнал, что примерно 5 моих изначальных вариантов не работают, когда начал проверять на больших объемах данных)